# **QLoRA Sentiment Fine-Tuning**

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes pandas certifi huggingface_hub --quiet
import torch
print(torch.cuda.is_available())  # should print: True


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
True


Authenticate Hugging Face

In [ ]:
from huggingface_hub import login
login("hf_token")

Load & Format Dataset

In [ ]:
import pandas as pd
import json

# Load your CSV file
df = pd.read_csv("Womens Clothing E-Commerce Reviews.csv")
df = df.dropna(subset=["Review Text"])

# Instruction formatting
def convert(row):
    sentiment = "positive" if row["Rating"] >= 4 else "negative"
    return {
        "instruction": f"Classify the sentiment of this review: '{row['Review Text']}'",
        "input": "",
        "output": sentiment
    }

samples = df.apply(convert, axis=1).dropna().tolist()
subset = samples[:500]  # Reduce size for stability with QLoRA

# Save as JSON
with open("sentiment_instructions.json", "w") as f:
    json.dump(subset, f, indent=2)


Load & Tokenize Dataset

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

df_json = pd.read_json("sentiment_instructions.json")
dataset = Dataset.from_pandas(df_json)

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token="hf_token")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    prompts = [i + " " + x for i, x in zip(example["instruction"], example["input"])]
    inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=128)["input_ids"]
    import torch
    inputs["labels"] = torch.tensor(labels)
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=16)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Load Model with 4-bit QLoRA

In [ ]:

from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token="hf_token",
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)




/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Apply LoRA Configuration

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


Train the Model

In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable Weights & Biases

training_args = TrainingArguments(
    output_dir="./qlora_sentiment",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    gradient_checkpointing=False,  # Disable to avoid grad_fn error
    optim="paged_adamw_8bit"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,6.507100
20,2.163800
30,0.834700
40,0.751900
50,1.136600
60,1.303900
70,0.699600
80,0.752000
90,0.913400
100,0.816600


TrainOutput(global_step=750, training_loss=0.7875858065287272, metrics={'train_runtime': 777.0633, 'train_samples_per_second': 1.93, 'train_steps_per_second': 0.965, 'total_flos': 8195406299136000.0, 'train_loss': 0.7875858065287272, 'epoch': 3.0})

Evaluate the Model

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Classify the sentiment of this review: 'I love how soft and flattering this shirt feels!'\nSentiment:"

response = pipe(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)


print(response[0]["generated_text"])


Device set to use cuda:0


Classify the sentiment of this review: 'I love how soft and flattering this shirt feels!'
Sentiment: positive


In [ ]:
prompt = "Classify the sentiment of this review: 'I dislike how soft and flattering this shirt feels!'\nSentiment:"

response = pipe(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)
print(response[0]["generated_text"])


Classify the sentiment of this review: 'I dislike how soft and flattering this shirt feels!'
Sentiment: negative
